# Vorlesung 1-1 (Montag 18.10.21)

## (1.7) Beispiel: sechs Varianten des Matrixprodukts $C = AB$

### Cheat-Sheet: Grundoperationen des Matrizenkalküls in Julia


| Bedeutung              | Formel                          | Julia                                           |
| :--------------------: | :-----------------------------: | :---------------------------------------------: |
| Komponente von $x$     | $\xi_k$                         | `x[k]`                                          |
| Komponente von $A$     | $\alpha_{jk}$                   | `A[j,k]`                                        |
| Spaltenvektor von $A$  | $a^k$                           | `A[:,k]`                                        |
| Zeilenvektor von $A$   | $a_j'$                          | `conj(A[j,:]')`, äquivalent `transpose(A[j,:])` |
| Untermatrix von $A$    | $$(\alpha_{jk})_{j=m:p,k=n:l}$$ | `A[m:p,n:l]`                                    |
| Adjungierte von $A$    | $A'$                            | `A'`                                            |
| Matrixprodukt          | $AB$                            | `A*B`                                           |
| Identität              | $I \in {\mathbb K}^{m\times m}$ | `Matrix{K}(I,m,m)`                              |
| Nullmatrix             | $0 \in {\mathbb K}^{m\times n}$ | `zeros(K,m,n)`                                  |

hierbei ist `K = typeof(α)` der für Skalare `α` verwendete Datentyp als Proxy für den  Grundkörper ${\mathbb K}$.

### Vorbereitungen

In [1]:
using LinearAlgebra

# die Testmatrizen

m = 2000
A = randn(m,m)
B = randn(m,m)

# lade die Benchmark-Umgebung
# import Pkg; Pkg.add("BenchmarkTools") # ohne Kommentarzeichen beim ersten Lauf 

using BenchmarkTools

### Variante 1: spaltenweise
$$
A,\qquad B = 
\begin{pmatrix}
| &  & | \\
b^1 &  \!\!\!\cdots\!\!\! & b^p \\
| &  & | 
\end{pmatrix},\qquad
C = \begin{pmatrix}
| &  & | \\
Ab^1 &  \!\!\!\cdots\!\!\! & Ab^p \\
| &  & | 
\end{pmatrix}
$$

In [2]:
function P_spaltenweise(A,B)
    m, n = size(A)
    n, p = size(B)
    C = zeros(m,p)
    for l=1:p
        @views C[:,l] = A*B[:,l]     # '@views' vermeidet unnötiges Umkopieren
    end
    return C
end

t_spaltenweise = @benchmark C = P_spaltenweise(A,B)

BenchmarkTools.Trial: 4 samples with 1 evaluation.
 Range (min … max):  1.457 s …  1.473 s  ┊ GC (min … max): 0.27% … 0.11%
 Time  (median):     1.468 s             ┊ GC (median):    0.07%
 Time  (mean ± σ):   1.467 s ± 6.700 ms  ┊ GC (mean ± σ):  0.11% ± 0.11%

  █                                    █    █            █  
  █▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  1.46 s        Histogram: frequency by time        1.47 s <

 Memory estimate: 61.28 MiB, allocs estimate: 2002.

### Variante 2: zeilenweise

$$
A = \begin{pmatrix}
—  a_1'  — \\
 \vdots   \\
—  a_m'  —
\end{pmatrix},\qquad
B,\qquad
C = \begin{pmatrix}
—  a_1'\,B  — \\
 \vdots   \\
—  a_m'\,B  —
\end{pmatrix}
$$

In [3]:
function P_zeilenweise(A,B)
    m, n = size(A)
    n, p = size(B)
    C = zeros(m,p)
    for j=1:m
        @views C[j,:] = conj(A[j,:]')*B
    end
    return C
end

t_zeilenweise = @benchmark C = P_zeilenweise(A,B)

BenchmarkTools.Trial: 4 samples with 1 evaluation.
 Range (min … max):  1.377 s …   1.431 s  ┊ GC (min … max): 0.05% … 0.04%
 Time  (median):     1.408 s              ┊ GC (median):    0.05%
 Time  (mean ± σ):   1.406 s ± 22.481 ms  ┊ GC (mean ± σ):  0.08% ± 0.06%

  █                         █           █                 █  
  █▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  1.38 s         Histogram: frequency by time        1.43 s <

 Memory estimate: 61.28 MiB, allocs estimate: 2002.

### Variante 3: innere Produkte

$$
A = \begin{pmatrix}
—  a_1'  — \\
 \vdots   \\
—  a_m'  —
\end{pmatrix},\qquad
B = 
\begin{pmatrix}
| &  & | \\
b^1 &  \!\!\!\cdots\!\!\! & b^p \\
| &  & | 
\end{pmatrix},\qquad
C = \begin{pmatrix}
a_1'\, b^1 & \cdots & a_1'\, b^p \\
\vdots &  & \vdots\\
a_m'\, b^1 & \cdots & a_m'\, b^p
\end{pmatrix}
$$

In [4]:
function P_innere_produkte(A,B)
    m, n = size(A)
    n, p = size(B)
    C = zeros(m,p)
    for j=1:m
        for l=1:p
            @views C[j,l] = conj(A[j,:]')*B[:,l]
        end
    end
    return C
end

t_innere_produkte = @benchmark C = P_innere_produkte(A,B)

BenchmarkTools.Trial: 1 sample with 1 evaluation.
 Single result which took 12.703 s (0.00% GC) to evaluate,
 with a memory estimate of 30.52 MiB, over 2 allocations.

### Variante 4: äußere Produkte

$$
A = \begin{pmatrix}
| &  & | \\
a^1 &  \!\!\!\cdots\!\!\! & a^n \\
| &  & | 
\end{pmatrix},\qquad
B = \begin{pmatrix}
—  b_1'  — \\
 \vdots   \\
—  b_n'  —
\end{pmatrix},\qquad
C =  \sum_{k=1}^n a^k \, b_k'
$$

In [5]:
function P_äußere_produkte(A,B)
    m, n = size(A)
    n, p = size(B)
    C = zeros(m,p)  
    for k=1:n
        @views C .+= A[:,k].*conj(B[k,:]')
    end
    return C
end

t_äußere_produkte = @benchmark C = P_äußere_produkte(A,B)

BenchmarkTools.Trial: 1 sample with 1 evaluation.
 Single result which took 6.821 s (0.00% GC) to evaluate,
 with a memory estimate of 30.52 MiB, over 2 allocations.

In Zeile 6 wird das äußere Produkt als Broadcast-Produkt `.*` („jede Komponente des Spaltenvektors wird jeweils mit dem gesamten Zeilenvektor multipliziert“) ausgeführt, was deutlich langsamer als ein optimiertes äußeres Produkt ist. Wir können Julia aber durch direkten Aufruf der BLAS-Routine `ger` anweisen, stattdessen wirklich nur ein äußeres Produkt zu berechnen. Das '`!`' in `ger!` zeigt dabei an, dass Julia einen Update direkt auf dem aktuellen Speichplatz der Variablen `C` ausführt, diese Matrix wird also _in situ_ (d.h. „an Ort und Stelle“, ohne zusätzlichen Speicher anzulegen) überschrieben:

In [6]:
?BLAS.ger! # suche nach BLAS-Syntax für äußere Produkte

```
ger!(alpha, x, y, A)
```

Rank-1 update of the matrix `A` with vectors `x` and `y` as `alpha*x*y' + A`.


In [7]:
function P_äußere_produkte_BLAS2(A,B)
    m, n = size(A)
    n, p = size(B)
    C = zeros(m,p)  
    for k=1:n
        @views BLAS.ger!(1.0,A[:,k],B[k,:],C)
    end
    return C
end

t_äußere_produkte_BLAS2 = @benchmark C = P_äußere_produkte_BLAS2(A,B)

BenchmarkTools.Trial: 2 samples with 1 evaluation.
 Range (min … max):  2.957 s …   3.077 s  ┊ GC (min … max): 0.02% … 0.00%
 Time  (median):     3.017 s              ┊ GC (median):    0.01%
 Time  (mean ± σ):   3.017 s ± 84.985 ms  ┊ GC (mean ± σ):  0.01% ± 0.01%

  █                                                       █  
  █▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  2.96 s         Histogram: frequency by time        3.08 s <

 Memory estimate: 30.52 MiB, allocs estimate: 2.

### Variante 5: komponentenweise

$$
A = (\alpha_{jk})_{j=1:m,k=1:n},\qquad
B = (\beta_{kl})_{k=1:n,l=1:p},\qquad
C = \left(\sum_{k=1}^n \alpha_{jk}\beta_{kl}\right)_{j=1:m,l=1:p}
$$

In [8]:
function P_komponentenweise(A,B)
    m, n = size(A)
    n, p = size(B)
    C = zeros(m,p)
    for j=1:m
        for l=1:p
            for k=1:n
                @inbounds C[j,l] += A[j,k]*B[k,l] # '@inbounds' verhindert überflüssige Indexprüfung
            end
        end
    end
    return C
end

t_komponentenweise = @benchmark C = P_komponentenweise(A,B)

BenchmarkTools.Trial: 1 sample with 1 evaluation.
 Single result which took 12.834 s (0.00% GC) to evaluate,
 with a memory estimate of 30.52 MiB, over 2 allocations.

### Variante 6: direkt (BLAS3)

$$
A, \qquad
B, \qquad
C = AB
$$

In [9]:
t_BLAS3 = @benchmark C = A*B

BenchmarkTools.Trial: 44 samples with 1 evaluation.
 Range (min … max):  104.537 ms … 141.660 ms  ┊ GC (min … max): 0.00% … 5.82%
 Time  (median):     112.110 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   113.869 ms ±   7.598 ms  ┊ GC (mean ± σ):  2.27% ± 3.19%

    █    ▄█▄▁ ▁▄▁ ▁  ▁                                           
  ▆▆█▁▆▆▆████▁███▆█▁▁█▁▆▆▆▆▁▁▆▆▆▆▁▁▁▁▁▁▁▁▁▁▁▆▁▁▁▆▁▁▁▁▁▁▁▁▁▁▁▁▁▆ ▁
  105 ms           Histogram: frequency by time          142 ms <

 Memory estimate: 30.52 MiB, allocs estimate: 2.

### Auswertung der Laufzeiten

In [10]:
messungen = [t_komponentenweise,
    t_innere_produkte,
    t_äußere_produkte_BLAS2,
    t_spaltenweise,
    t_zeilenweise,
    t_BLAS3]

sec = 1e-9 # Umrechung von Nanosekunden in Sekunden
dig = 4    # Rundung auf zwei Nachkommastellen
laufzeiten = round.(time.(mean.(messungen))sec, digits=dig) 

faktoren = round.(laufzeiten/minimum(laufzeiten))

println("Laufzeiten: ", laufzeiten)
println("Faktoren: ", faktoren)

Laufzeiten: [12.8335, 12.7027, 3.0167, 1.4668, 1.406, 0.1139]
Faktoren: [113.0, 112.0, 26.0, 13.0, 12.0, 1.0]


| Methode          | # for-Schleifen  | BLAS-Level | Laufzeit [sec] |  Verlangsamungsfaktor |
| :-:              |:-:|:-:|   --:|  --:|
| komponentenweise | 3 | 0 | 12.8 | 110 | 
| innere Produkte  | 2 | 1 | 12.7 | 110 | 
| äußere Produkte  | 1 | 2 | 3.02 |  26 | 
| spaltenweise     | 1 | 2 | 1.47 |  13 |
| zeilenweise      | 1 | 2 | 1.41 |  12 |
| direkt           | 0 | 3 | 0.11 |   1 |